In [1]:
from http import HTTPStatus
import dashscope
import json

import os

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 设置环境变量
os.environ['http_proxy'] = 'http://gpu013:7890'
os.environ['https_proxy'] = 'http://gpu013:7890' # 如果你也需要设置HTTPS代理

# 验证环境变量是否设置成功
print(os.environ['http_proxy'])

/mnt/cephfs/home/chendeyu/anaconda3/envs/KG_RAG/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


http://gpu013:7890


In [ ]:
import os
from pathlib import Path
import sys

# 获取当前工作目录的绝对路径
current_dir = Path.cwd()

# 将配置文件的路径添加到系统路径中
config_path = os.path.join(current_dir, "key_config.py")
if config_path not in sys.path:
    sys.path.append(config_path)

# # 导入 API 密钥
from key_config import OPENAI_KEY
from key_config import DASHSCOPE_KEY

# # 现在可以使用 API_KEY 变量了
print(OPENAI_KEY)
print(DASHSCOPE_KEY)


In [2]:
from sentence_transformers import SentenceTransformer, util

# 检查CUDA是否可用
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA is available! Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU instead.")

# 加载模型
model = SentenceTransformer('all-MiniLM-L6-v2')
model = model.to(device)

CUDA is available! Using GPU: NVIDIA TITAN Xp


In [3]:
dashscope.api_key = DASHSCOPE_KEY

# system_prompt = '''Now that you are now an expert in summarizing summaries, you are now given a abstract of an article, please extract from it which field it belongs to (like continuous learning, large language models, reinforcement learning, etc.), problems, methods and effects, and return it in the following json format {"field":, "problems":, "methods":, "effects":}Please return only the {...} not include json at the beginning and nothing else'''
abstract_prompt = '''Now that you are now an expert in summarizing summaries, you are now given a abstract of an article, please extract from it which field it belongs to (like continuous learning, large language models, reinforcement learning, etc.), problems, methods and effects, and return it in the following json format {"field":"...", "problems":"...", "methods":"...", "effects":"..."}.'''


def qwen_get_response(system_prompt, question):
    messages = [{'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': question}]

    response = dashscope.Generation.call(
        dashscope.Generation.Models.qwen_plus,
        messages=messages,
        result_format='message',  # set the result to be "message" format.
    )
    if response.status_code == HTTPStatus.OK:
        print(response)
        formatted_response = json.dumps(response, indent=4, ensure_ascii=False)
        print(formatted_response)
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    return formatted_response

In [4]:
def qwen_72b_chat_entity_extractor(text):
    entity_prompt = '''Now that you are an entity recognition expert, you are now given a text, you need to identify all the entities of the text,
    and return Entities in the following json format: \nEntities:<List of extracted entities>\n.'''
    resp = qwen_get_response(entity_prompt, text)
    entity_dict = json.loads(resp)
    print(entity_dict)
    return entity_dict

In [5]:
# 更新为包含意思相近的句子和文本
json_data = {
    "entities": {
        "Traveling helps you discover who you are and lets you experience the vastness of this world.":{},  # Similar to entity1
        "Reaching the summit of a mountain offers an exhilarating sense of achievement.":{},  # Similar to entity2
        "There is a special kind of calm in the early hours of the morning in a lush forest.":{},  # Similar to entity3
        "Diving into learning Python has unlocked new creative potentials for me.":{},  # Similar to entity4
        "The act of preparing a meal with family is a source of joy and brings everyone closer.":{},  # Similar to entity5
        "Immersing myself in the pages of a historical novel allows me to live in a different era.":{},  # Similar to entity6
        "Finding myself through the melodies I play on the piano is a profound experience.":{},  # Similar to entity7
        "The joy of watching my garden grow after days of care is incredibly fulfilling.":{}  # Similar to entity8
    }
}

texts = [
    "self-discovery travel experience",
    "mountain climbing achievement",
    "morning forest tranquility",
    "python coding creativity",
    "family cooking bonding",
    "historical fiction immersion",
    "piano music self-expression",
    "gardening nurturing fulfillment"
]



# 计算相似度的函数,返回文本之间的余弦相似度
for text in texts:
    print(f"文本: '{text}'")
    scores = []
    text_embedding = model.encode(text, convert_to_tensor=True)
    for entity_id, entity_text in json_data["entities"].items():
        entity_embedding = model.encode(entity_id, convert_to_tensor=True)
        similarity_score = util.pytorch_cos_sim(text_embedding, entity_embedding).item()
        scores.append((entity_id, similarity_score))

    # 根据相似度分数排序并选择最相似的3个实体
    scores.sort(key=lambda x: x[1], reverse=True)
    top_matches = scores[:3]

    # 打印匹配结果及其相似度分数
    for match in top_matches:
        print(f"匹配到 '{match[0]}', 相似度分数: {match[1]:.4f}")
    print("\n" + "="*80 + "\n")


文本: 'self-discovery travel experience'
匹配到 'Traveling helps you discover who you are and lets you experience the vastness of this world.', 相似度分数: 0.6171
匹配到 'Reaching the summit of a mountain offers an exhilarating sense of achievement.', 相似度分数: 0.3315
匹配到 'Immersing myself in the pages of a historical novel allows me to live in a different era.', 相似度分数: 0.3158


文本: 'mountain climbing achievement'
匹配到 'Reaching the summit of a mountain offers an exhilarating sense of achievement.', 相似度分数: 0.7586
匹配到 'Traveling helps you discover who you are and lets you experience the vastness of this world.', 相似度分数: 0.1685
匹配到 'The joy of watching my garden grow after days of care is incredibly fulfilling.', 相似度分数: 0.1483


文本: 'morning forest tranquility'
匹配到 'There is a special kind of calm in the early hours of the morning in a lush forest.', 相似度分数: 0.6990
匹配到 'Immersing myself in the pages of a historical novel allows me to live in a different era.', 相似度分数: 0.2298
匹配到 'The joy of watching my gard